<a href="https://colab.research.google.com/github/LuisCGit/HipHopPopularity/blob/main/pretrained_model_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import random 

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install timm==0.4.5 pytorch-lightning optuna

     |████████████████████████████████| 287 kB 6.8 MB/s 
     |████████████████████████████████| 582 kB 48.3 MB/s 
     |████████████████████████████████| 308 kB 55.0 MB/s 
     |████████████████████████████████| 408 kB 49.3 MB/s 
     |████████████████████████████████| 136 kB 52.0 MB/s 
     |████████████████████████████████| 596 kB 42.0 MB/s 
     |████████████████████████████████| 1.1 MB 51.2 MB/s 
     |████████████████████████████████| 81 kB 7.0 MB/s 
     |████████████████████████████████| 210 kB 80.7 MB/s 
     |████████████████████████████████| 271 kB 69.7 MB/s 
     |████████████████████████████████| 144 kB 73.2 MB/s 
     |████████████████████████████████| 94 kB 2.6 MB/s 
     |████████████████████████████████| 78 kB 8.2 MB/s 
     |████████████████████████████████| 49 kB 7.1 MB/s 
     |████████████████████████████████| 146 kB 70.6 MB/s 
     |████████████████████████████████| 113 kB 77.6 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=111

In [15]:
import pickle
import numpy as np
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, r2_score, mean_absolute_error, mean_squared_error
import torch
import torch.nn as nn
from torch.nn import functional as F
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks.model_checkpoint import ModelCheckpoint
import optuna
from drive.MyDrive.MIT.HODL.ssast.src.models.ast_models import ASTModel

In [39]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchaudio import transforms

In [18]:
#!wget -O mel_spec_datadict https://www.dropbox.com/s/nrebpbe5g97sptu/mel_spec_datadict?dl=0
!wget -O mel_spec_datadict_all https://www.dropbox.com/s/e9l6xid06080g6z/mel_spec_datadict_all?dl=0

--2022-05-03 19:01:00--  https://www.dropbox.com/s/e9l6xid06080g6z/mel_spec_datadict_all?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/e9l6xid06080g6z/mel_spec_datadict_all [following]
--2022-05-03 19:01:01--  https://www.dropbox.com/s/raw/e9l6xid06080g6z/mel_spec_datadict_all
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc98b85cb1d4ff44bbdb56045309.dl.dropboxusercontent.com/cd/0/inline/BkmL_3S92VTIc_LeXdXFVTHMFN5BNi2s2HQAQh5OkfYLK6ZpVSf8SdbyOBnQFEGcUFUpGYFLEvGfWGYIBcmM-oyXOSPU-g4CfM9S9BsEfJo17GyrXjhWHsml6oTWVLzsEV3bAthrJxzbcXyh12EVNYDdehTrCizp7grLeTm8pjgSyg/file# [following]
--2022-05-03 19:01:01--  https://uc98b85cb1d4ff44bbdb56045309.dl.dropboxusercontent.com/cd/0/inline/BkmL_3S92VTIc_LeXdXFVTHMFN5BNi2s2HQA

In [20]:
with open('mel_spec_datadict_all', 'rb') as f:
    mel_spec_datadict = pickle.load(f)
# with open('mel_spec_datadict_all', 'rb') as f:

In [22]:
!wget -O raw_datadict_all https://www.dropbox.com/s/yfi64jhyk9wzv6l/raw_datadict_all?dl=0

--2022-05-03 19:03:56--  https://www.dropbox.com/s/yfi64jhyk9wzv6l/raw_datadict_all?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.1.18, 2620:100:6030:18::a27d:5012
Connecting to www.dropbox.com (www.dropbox.com)|162.125.1.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/yfi64jhyk9wzv6l/raw_datadict_all [following]
--2022-05-03 19:03:57--  https://www.dropbox.com/s/raw/yfi64jhyk9wzv6l/raw_datadict_all
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1ca114251c79be673614dd4fff.dl.dropboxusercontent.com/cd/0/inline/Bkl3SjPYcrFcFRzQhSj3srZU_L7jkrvbOMuf8vAVjzet2H3R818eE8yw34APtpO4MbTRuuagCx1YQ5eivHRkBwOmgmZtmjSoWIPvz2WQMjfvzfDoxfDfmpLX93h-994kSNqX3i6sEzRTs73-DQ-Qc4J0iDuhpnN_OhkJu-NJy2a6rA/file# [following]
--2022-05-03 19:03:57--  https://uc1ca114251c79be673614dd4fff.dl.dropboxusercontent.com/cd/0/inline/Bkl3SjPYcrFcFRzQhSj3srZU_L7jkrvbOMuf8vAVjzet2H3R81

In [23]:
with open('raw_datadict_all', 'rb') as f:
  raw_datadict_all = pickle.load(f)

In [29]:
X_train, y_train = mel_spec_datadict['X_train']
X_val, y_val = mel_spec_datadict['X_holdout']
X_test, y_test = mel_spec_datadict['X_test']

X_train = mel_spec_datadict['X_train'][0]
X_val = mel_spec_datadict['X_holdout'][0]
X_test = mel_spec_datadict['X_test'][0]

y_train = raw_datadict_all['X_train'][0]['popGreaterThan50']
y_val = raw_datadict_all['X_holdout'][0]['popGreaterThan50']
y_test = raw_datadict_all['X_test'][0]['popGreaterThan50']
y_train2 = raw_datadict_all['X_train'][0]['popularity']
y_val2 = raw_datadict_all['X_holdout'][0]['popularity']
y_test2 = raw_datadict_all['X_test'][0]['popularity']

In [30]:
pl.seed_everything(0, workers=True)

Global seed set to 0


0

In [31]:
# hyperparameters
NB_EPOCHS = 20
PATIENCE = 6
DOWNSAMPLING_FACTOR = 10
INPUT_TDIM = int(X_train.shape[2] / DOWNSAMPLING_FACTOR)
INPUT_FDIM = X_train.shape[1]
TASK = 'classification'
MONITOR = 'val_loss'
MONITOR_HYPERPARAM = 'val_auc'

BATCH_SIZE = 32
LEARNING_RATE = 3e-4
DROPOUT_PROB = 0.0
USE_CLS_POOLING = True
NB_LAYERS_TO_FREEZE = 12
GRAD_ACC = 1

In [32]:
# downsampling
indices_to_isolate = [k * DOWNSAMPLING_FACTOR for k in range(INPUT_TDIM)]
len(indices_to_isolate)

129

In [125]:
# data
# X_train_downsampled = X_train[:,:,indices_to_isolate,:]
# X_train_normalized = (X_train_downsampled - X_train_downsampled.mean(0)) / X_train_downsampled.std(0)
# X_val_downsampled = X_val[:,:,indices_to_isolate,:]
# X_val_normalized = (X_val_downsampled - X_train_downsampled.mean(0)) / X_train_downsampled.std(0)
# X_test_downsampled = X_test[:,:,indices_to_isolate,:]
# X_test_normalized = (X_test_downsampled - X_train_downsampled.mean(0)) / X_train_downsampled.std(0)

transform = nn.Sequential(transforms.FrequencyMasking(freq_mask_param=80).to(device), transforms.TimeMasking(time_mask_param=80).to(device))
transform = transform.to(device)

X_train_ = transform(torch.from_numpy(X_train_normalized).to(torch.float16).squeeze(-1).to(device))
X_val_ = transform(torch.from_numpy(X_val_normalized).to(torch.float16).squeeze(-1))
X_test_ = transform(torch.from_numpy(X_test_normalized).to(torch.float16).squeeze(-1))
y_train_ = torch.from_numpy(y_train.values).to(torch.float16)
y_val_ = torch.from_numpy(y_val.values).to(torch.float16)
y_test_ = torch.from_numpy(y_test.values).to(torch.float16)



class SpectDataset(torch.utils.data.TensorDataset):
  def __getitem__(self, index):
    #tens = tuple(tensor[index] for tensor in self.tensors)
    #print("ken")
    #tens = tuple(transform(t) for t in tens)
    # out = tuple(tensor[index] for tensor in self.tensors)
    # for tensor
    # for thing in out:
    #   print("shape ting: ", thing.shape)
    print("shape self.tens: ", self.tensors[0].shape)
    return tuple(transform(tensor)[index] for tensor in self.tensors)
train = SpectDataset(X_train_, y_train_)
val = SpectDataset(X_val_, y_val_)
test = SpectDataset(X_test_, y_test_)
# loaders
train_loader = torch.utils.data.DataLoader(train, sampler=torch.utils.data.RandomSampler(train), batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val, sampler=torch.utils.data.SequentialSampler(val), batch_size=BATCH_SIZE*2)
test_loader = torch.utils.data.DataLoader(test, sampler=torch.utils.data.SequentialSampler(test), batch_size=BATCH_SIZE*2)

RuntimeError: ignored

In [34]:
def freq_mask(spec, F=30, num_masks=1, replace_with_zero=False):
    cloned = spec.clone()
    num_mel_channels = cloned.shape[1]
    
    for i in range(0, num_masks):        
        f = random.randrange(0, F)
        f_zero = random.randrange(0, num_mel_channels - f)

        # avoids randrange error if values are equal and range is empty
        if (f_zero == f_zero + f): return cloned

        mask_end = random.randrange(f_zero, f_zero + f) 
        if (replace_with_zero): cloned[0][f_zero:mask_end] = 0
        else: cloned[0][f_zero:mask_end] = cloned.mean()
    
    return cloned

def time_mask(spec, T=40, num_masks=1, replace_with_zero=False):
    cloned = spec.clone()
    len_spectro = cloned.shape[2]
    
    for i in range(0, num_masks):
        t = random.randrange(0, T)
        t_zero = random.randrange(0, len_spectro - t)

        # avoids randrange error if values are equal and range is empty
        if (t_zero == t_zero + t): return cloned

        mask_end = random.randrange(t_zero, t_zero + t)
        if (replace_with_zero): cloned[0][:,t_zero:mask_end] = 0
        else: cloned[0][:,t_zero:mask_end] = cloned.mean()
    return cloned

In [35]:
class CustomASTModel(pl.LightningModule):

  def __init__(
      self,
      input_fdim=INPUT_FDIM,
      input_tdim=INPUT_TDIM,
      use_small=True,
      use_cls_pooling=USE_CLS_POOLING,
      dropout_prob=DROPOUT_PROB,
      task='classification',
      regression_threshold=1.0,
      nb_layers_to_freeze=NB_LAYERS_TO_FREEZE,
      lr=LEARNING_RATE
  ):
    super().__init__()
    self.lr = lr
    self.dropout_prob = dropout_prob
    self.regression_threshold = regression_threshold

    if use_cls_pooling:
      self.pooling_type = 'ft_cls'
    else:
      self.pooling_type = 'ft_avgtok'

    assert(task in ['classification','regression'])
    self.task = task

    if use_small:
      self.model = ASTModel(
          label_dim=1,    # no impact
          fshape=16, 
          tshape=16, 
          fstride=10, 
          tstride=10,
          input_fdim=input_fdim, 
          input_tdim=input_tdim, 
          model_size='small',
          pretrain_stage=False, 
          load_pretrained_mdl_path=(
              '/content/drive/MyDrive/MIT/HODL/ssast/pretrained_model/SSAST-Small-Patch-400.pth'
          )
      )
    else:
       self.model = ASTModel(
          label_dim=1,
          fshape=16, 
          tshape=16, 
          fstride=10, 
          tstride=10,
          input_fdim=input_fdim, 
          input_tdim=input_tdim, 
          model_size='base',
          pretrain_stage=False, 
          load_pretrained_mdl_path=(
              '/content/drive/MyDrive/MIT/HODL/ssast/pretrained_model/SSAST-Base-Patch-400.pth'
          )
       ) 
    self.cls_shape = self.model.v.blocks[-1].mlp.fc2.out_features
    mlp_head = nn.Sequential(
        nn.LayerNorm(self.cls_shape),
        nn.Linear(in_features=self.cls_shape, out_features=int(self.cls_shape / 2), bias=True),
        nn.Dropout(self.dropout_prob),
        nn.Linear(in_features=int(self.cls_shape / 2), out_features=1, bias=True),
    )
    self.model.mlp_head = mlp_head

    if nb_layers_to_freeze > 0:
      assert(nb_layers_to_freeze-1 in range(len(self.model.v.blocks)))
      for i,block in enumerate(self.model.v.blocks):
        if i < nb_layers_to_freeze:
          for param in block.parameters():
            param.requires_grad = False
        else:
          break


  def forward(self, x):
    x = time_mask(freq_mask(x, num_masks=2), num_masks=2)
    h = self.model(x, task=self.pooling_type)
    return x


  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
    return optimizer


  def training_step(self, train_batch, batch_idx):
    x, y = train_batch
    x = x.cuda(0)
    y = y.cuda(0).unsqueeze(1)
    y_hat = self.model(x, task=self.pooling_type)

    if self.task == 'classification':
      loss = F.binary_cross_entropy_with_logits(y_hat, y)
      sigmoid = torch.sigmoid(y_hat)
      auc = roc_auc_score(y.cpu().detach().numpy(), sigmoid.cpu().detach().numpy())
      threshold = torch.tensor([0.5]).to(torch.float16).cuda(0)
      results = (sigmoid > threshold).to(torch.float16)
      acc = accuracy_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      f1 = f1_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      self.log('train_auc', auc)
      self.log('train_acc', acc)
      self.log('train_f1', acc)

    elif self.task == 'regression':
      # y_hat = torch.relu(y_hat)   # we are trying to predict something positive so using relu makes sense
      y_hat = torch.clip(y_hat, min=np.float16(0.0), max=np.float16(self.regression_threshold))
      loss = F.mse_loss(y_hat, y)
      rmse = np.sqrt(mean_squared_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()))
      mae = mean_absolute_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      r2 = r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      self.log('train_rmse', rmse)
      self.log('train_mae', mae)
      self.log('train_r2', r2)

    self.log('train_loss', loss)
    return loss


  def validation_step(self, val_batch, batch_idx):
    x, y = val_batch
    x = x.cuda(0)
    y = y.cuda(0).unsqueeze(1)
    y_hat = self.model(x, task=self.pooling_type)

    if self.task == 'classification':
      loss = F.binary_cross_entropy_with_logits(y_hat, y)
      sigmoid = torch.sigmoid(y_hat)
      auc = roc_auc_score(y.cpu().detach().numpy(), sigmoid.cpu().detach().numpy())
      threshold = torch.tensor([0.5]).to(torch.float16).cuda(0)
      results = (sigmoid > threshold).to(torch.float16)
      acc = accuracy_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      f1 = f1_score(y.cpu().detach().numpy(), results.cpu().detach().numpy())
      self.log('val_auc', auc)
      self.log('val_acc', acc)
      self.log('val_f1', acc)

    elif self.task == 'regression':
      # y_hat = torch.relu(y_hat)   # we are trying to predict something positive so using relu makes sense
      y_hat = torch.clip(y_hat, min=np.float16(0.0), max=np.float16(self.regression_threshold))
      loss = F.mse_loss(y_hat, y)
      rmse = np.sqrt(mean_squared_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy()))
      mae = mean_absolute_error(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      r2 = r2_score(y.cpu().detach().numpy(), y_hat.cpu().detach().numpy())
      self.log('val_rmse', rmse)
      self.log('val_mae', mae)
      self.log('val_r2', r2)

    self.log('val_loss', loss)

# some simple tests

In [46]:
class SpectDataset(torch.utils.data.DataLoader):
  def __init__(self):
    super().__init__
  def __getitem__(self, idx):
    if torch.is_tensor(idx):
        idx = idx.tolist()

        img_name = os.path.join(self.root_dir,
                                self.landmarks_frame.iloc[idx, 0])
        image = io.imread(img_name)
       
        ### SOME DATA MANIPULATION HERE ###

        sample = {'image': image, 'landmarks': landmarks}
        if self.transform:
            sample = self.transform(sample)

        return sample


In [51]:
train_loader.dataset.__getitem__(9)

(tensor([[-0.0026, -0.8188, -0.6235,  ...,  0.1841, -0.4519, -0.4561],
         [ 0.0114, -1.0859, -0.7720,  ...,  0.3545, -0.6689, -0.8516],
         [ 0.1675, -0.3538, -0.1170,  ...,  0.2064,  0.1594,  0.0586],
         ...,
         [-0.3284,  0.3184,  1.5928,  ...,  1.6836,  1.0801,  1.1865],
         [-0.0758,  0.0768,  1.4512,  ...,  1.5566,  0.9556,  1.1738],
         [ 0.4646,  0.0022,  1.0381,  ...,  1.6123,  0.8916,  1.0459]],
        dtype=torch.float16), tensor(0., dtype=torch.float16))

In [52]:
train

In [76]:
# loaders
train_loader = torch.utils.data.DataLoader(train, sampler=torch.utils.data.RandomSampler(train), batch_size=BATCH_SIZE)
val_loader = torch.utils.data.DataLoader(val, sampler=torch.utils.data.SequentialSampler(val), batch_size=BATCH_SIZE*2)
test_loader = torch.utils.data.DataLoader(test, sampler=torch.utils.data.SequentialSampler(test), batch_size=BATCH_SIZE*2)

In [72]:
# model
model = CustomASTModel()

now load a SSL pretrained models from /content/drive/MyDrive/MIT/HODL/ssast/pretrained_model/SSAST-Small-Patch-400.pth
pretraining patch split stride: frequency=16, time=16
pretraining patch shape: frequency=16, time=16
pretraining patch array dimension: frequency=8, time=64
pretraining number of patches=512
fine-tuning patch split stride: frequncey=10, time=10
fine-tuning number of patches=144


In [116]:
# training
early_stop_callback = EarlyStopping(
    monitor=MONITOR, 
    min_delta=0.00, 
    patience=PATIENCE, 
    verbose=True, 
    mode=(
        "max" if MONITOR in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "min"
    )
)
trainer = pl.Trainer(
    gpus=1,
    limit_train_batches=1.0, 
    accumulate_grad_batches=GRAD_ACC,
    precision=16, 
    max_epochs=NB_EPOCHS, 
    callbacks=[early_stop_callback]
)
trainer.fit(model, train_loader, val_loader)

Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | ASTModel | 22.3 M
-----------------------------------
1.0 M     Trainable params
21.3 M    Non-trainable params
22.3 M    Total params
44.589    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

shape self.tens:  torch.Size([394, 128, 129])


RuntimeError: ignored

In [109]:
x = torch.rand((3008, 128, 129))

In [110]:
transform = nn.Sequential(transforms.FrequencyMasking(freq_mask_param=80), transforms.TimeMasking(time_mask_param=80))
transform(x).shape

torch.Size([3008, 128, 129])

In [ ]:
val_results = trainer.validate(model, val_loader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     Validate metric           DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
         val_acc            0.6529794692038057
         val_auc            0.6556286642504168
         val_f1             0.6529794692038057
        val_loss            0.6276569962501526
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=lightning_logs/

# hyperparameter tuning

In [ ]:
def objective(trial): 

  BATCH_SIZE = trial.suggest_int("batch_size", 1, 4)
  BATCH_SIZE *= 16
  LEARNING_RATE = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
  DROPOUT_PROB = trial.suggest_float("dropout_prob", 0.0, 0.5)
  USE_CLS_POOLING = trial.suggest_categorical("use_cls_pooling", [True, False])
  NB_LAYERS_TO_FREEZE = trial.suggest_int("nb_layers_to_freeze", 0, 12)
  GRAD_ACC = trial.suggest_int("grad_acc", 1, 2)

  train_loader = torch.utils.data.DataLoader(train, sampler=torch.utils.data.RandomSampler(train), batch_size=BATCH_SIZE)
  val_loader = torch.utils.data.DataLoader(val, sampler=torch.utils.data.SequentialSampler(val), batch_size=BATCH_SIZE)
  test_loader = torch.utils.data.DataLoader(test, sampler=torch.utils.data.SequentialSampler(test), batch_size=BATCH_SIZE)

  model = CustomASTModel(
      use_cls_pooling=USE_CLS_POOLING,
      dropout_prob=DROPOUT_PROB,
      nb_layers_to_freeze=NB_LAYERS_TO_FREEZE,
      lr=LEARNING_RATE      
  )

  early_stop_callback = EarlyStopping(
      monitor=MONITOR, 
      min_delta=0.00, 
      patience=PATIENCE, 
      verbose=True, 
      mode=(
          "max" if MONITOR in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "min"
      )
  )

  checkpoint_callback = ModelCheckpoint(
      save_top_k=1, 
      monitor=MONITOR_HYPERPARAM, 
      mode=(
          "max" if MONITOR_HYPERPARAM in ['val_auc', 'val_acc', 'val_f1', 'val_r2'] else "min"
      )
  )

  trainer = pl.Trainer(
      gpus=1,
      limit_train_batches=1.0, 
      accumulate_grad_batches=GRAD_ACC,
      precision=16, 
      max_epochs=NB_EPOCHS, 
      callbacks=[early_stop_callback, checkpoint_callback],
      enable_checkpointing=True,
      log_every_n_steps=16,
      # enable_progress_bar=False
  )

  trainer.fit(model, train_loader, val_loader)

  # val_results = trainer.validate(model, val_loader)
  
  # return val_results[0][MONITOR_HYPERPARAM]
  return checkpoint_callback.best_model_score

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=15, gc_after_trial=True)

[I 2022-05-03 05:32:00,945] A new study created in memory with name: no-name-ddc57a99-814d-48f9-9c7e-5f1112212add


now load a SSL pretrained models from /content/drive/MyDrive/HODL/ssast/pretrained_model/SSAST-Small-Patch-400.pth


Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1.0)` was configured so 100% of the batches per epoch will be used..
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type     | Params
-----------------------------------
0 | model | ASTModel | 22.3 M
-----------------------------------
17.0 M    Trainable params
5.3 M     Non-trainable params
22.3 M    Total params
44.589    Total estimated model params size (MB)


pretraining patch split stride: frequency=16, time=16
pretraining patch shape: frequency=16, time=16
pretraining patch array dimension: frequency=8, time=64
pretraining number of patches=512
fine-tuning patch split stride: frequncey=10, time=10
fine-tuning number of patches=144


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 6 records. Best score: 0.607. Signaling Trainer to stop.
[I 2022-05-03 05:55:10,720] Trial 14 finished with value: 0.7127921855921853 and parameters: {'batch_size': 1, 'learning_rate': 0.00012239861166378051, 'dropout_prob': 0.391279325562114, 'use_cls_pooling': True, 'nb_layers_to_freeze': 8, 'grad_acc': 1}. Best is trial 12 with value: 0.7211533030146442.


In [ ]:
study.best_trial

FrozenTrial(number=12, values=[0.7211533030146442], datetime_start=datetime.datetime(2022, 5, 3, 5, 50, 55, 767641), datetime_complete=datetime.datetime(2022, 5, 3, 5, 52, 18, 428308), params={'batch_size': 2, 'learning_rate': 7.83833546396864e-05, 'dropout_prob': 0.40324782505879203, 'use_cls_pooling': True, 'nb_layers_to_freeze': 8, 'grad_acc': 1}, distributions={'batch_size': IntUniformDistribution(high=4, low=1, step=1), 'learning_rate': LogUniformDistribution(high=0.01, low=1e-05), 'dropout_prob': UniformDistribution(high=0.5, low=0.0), 'use_cls_pooling': CategoricalDistribution(choices=(True, False)), 'nb_layers_to_freeze': IntUniformDistribution(high=12, low=0, step=1), 'grad_acc': IntUniformDistribution(high=2, low=1, step=1)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=12, state=TrialState.COMPLETE, value=None)

In [ ]:
'''
AUC: 0.7211533030146442
{
  'batch_size': 2, 
  'learning_rate': 7.83833546396864e-05, 
  'dropout_prob': 0.40324782505879203, 
  'use_cls_pooling': True, 
  'nb_layers_to_freeze': 8, 
  'grad_acc': 1
}
'''

In [ ]:
pretrained_model_path = '/content/drive/MyDrive/HODL/ssast/pretrained_model/SSAST-Small-Patch-400.pth'

In [ ]:
input_tdim = 128 # fine-tuning data length can be different with pretraining data length

In [120]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
import os

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
ast_mdl = ASTModel(label_dim=1,
             fshape=16, tshape=16, fstride=10, tstride=10,
             input_fdim=128, input_tdim=input_tdim, model_size='small',
             pretrain_stage=False, load_pretrained_mdl_path=pretrained_model_path).to(device)

RuntimeError: ignored

In [ ]:
aast_mdl.v.blocks[0].attn.proj.weight.dtype

torch.float32

In [ ]:
ast_mdl

ASTModel(
  (v): DistilledVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 384, kernel_size=(16, 16), stride=(10, 10))
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU()
          (fc2): Linear(in_features=1536, out_features=384, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (1): Block(
        (norm1): LayerNorm((384

In [ ]:
n = ast_mdl.v.blocks[-1].mlp.fc2.out_features

ast_mdl.mlp_head = nn.Sequential(
    nn.LayerNorm(n),
    nn.Linear(in_features=n, out_features=1, bias=True),
    nn.Sigmoid()
)#.to(torch.float)

In [ ]:
x = torch.from_numpy(np.random.rand(8, 128, 128)).float()

In [ ]:
ast_mdl(x, task='ft_cls')

tensor([[0.3713],
        [0.3795],
        [0.3953],
        [0.3442],
        [0.3849],
        [0.3578],
        [0.3586],
        [0.3877]], grad_fn=<SigmoidBackward0>)